<a href="https://colab.research.google.com/github/victorjoseij/LLM/blob/main/2348570_LLMLab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from datasets import load_dataset
dataset=load_dataset('imdb')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Take equal ratio of positive and negative reviews

In [ ]:
import random
postive = [x for x in dataset['train'] if x['label'] == 1][:2500]
negative = [x for x in dataset['train'] if x['label'] == 0][:2500]
train_dataset = postive + negative
random.shuffle(train_dataset)
positive_test = [x for x in dataset['test'] if x['label'] == 1][:2500]
negative_test = [x for x in dataset['test'] if x['label'] == 0][:2500]
test_dataset = positive_test + negative_test
random.shuffle(test_dataset)

Loading Tokenizer and BERT model

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Defining Train and Test data

In [ ]:
train_texts = [x['text'] for x in train_dataset]
train_labels = [x['label'] for x in train_dataset]
test_texts = [x['text'] for x in test_dataset]
test_labels = [x['label'] for x in test_dataset]

Tokenizing train and test test

In [ ]:
import tensorflow as tf
train_encodings=tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings=tokenizer(test_texts, truncation=True, padding=True, max_length=128)

Mapping train and test to labels

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
test_dataset=tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels))
train_dataset=train_dataset.shuffle(1000).batch(8).prefetch(tf.data.AUTOTUNE)
test_dataset=test_dataset.batch(8).prefetch(tf.data.AUTOTUNE)

Compiling the model

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Fitting the model

In [ ]:
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

Epoch 1/3
625/625 [==============================] - 277s 374ms/step - loss: 0.3960 - accuracy: 0.8168 - val_loss: 0.3327 - val_accuracy: 0.8554
Epoch 2/3
625/625 [==============================] - 233s 373ms/step - loss: 0.1925 - accuracy: 0.9278 - val_loss: 0.3772 - val_accuracy: 0.8672
Epoch 3/3
625/625 [==============================] - 201s 322ms/step - loss: 0.0779 - accuracy: 0.9748 - val_loss: 0.4186 - val_accuracy: 0.8650


Checking the Accuracy

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.2f}")

625/625 [==============================] - 52s 83ms/step - loss: 0.4186 - accuracy: 0.8650
Test Accuracy: 0.87


Sentiment Prediction function

In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=128)
    outputs = model(inputs)
    logits = outputs.logits
    predictions = tf.nn.softmax(logits, axis=-1)
    sentiment = tf.argmax(predictions, axis=1).numpy()[0]
    return sentiment

Predicting

In [ ]:
text = input("Enter text for sentiment analysis: ")
sentiment = predict_sentiment(text)
sentiment_label = "Positive" if sentiment == 1 else "Negative"
print(f"Sentiment: {sentiment_label}")

Enter text for sentiment analysis: i have no confidence to talk  with  strangers
Sentiment: Negative
